# calculate rotation and transposition matrix

In [7]:
data_folder = r'D:\Shiwei\20210423-E14_Forebrain_smRNA'
before_position_file = os.path.join(data_folder, '10x_positions.txt')
after_position_file = os.path.join(data_folder, '10x_positions_after.txt')


In [8]:
import numpy as np
import os, sys
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [9]:
R, T = align_manual_points(before_position_file, after_position_file, save=False)


In [10]:
R, T

(array([[ 0.99957146, -0.0292728 ],
        [ 0.0292728 ,  0.99957146]]), array([-874.28797339, -551.78765221]))

# transpose 60x positions

In [11]:
old_positions = np.loadtxt(os.path.join(data_folder, 'positions_all.txt'), delimiter=',')

In [12]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[ -408.14550219  -458.09279355]
 [ -401.67035977  -236.98758662]
 [ -395.19521736   -15.88237969]
 [ -388.72300222   205.12287009]
 [ -382.2478598    426.22807702]
 [ -375.77271739   647.33328395]
 [ -154.66751046   640.85814153]
 [ -161.14265288   419.7529346 ]
 [ -167.61779529   198.64772768]
 [ -174.09001043   -22.35752211]
 [ -180.56515284  -243.46272903]
 [ -187.04029526  -464.56793596]
 [   34.06491167  -471.04307838]
 [   40.54005409  -249.93787145]
 [   47.0151965    -28.83266452]
 [   53.48741164   192.17258526]
 [   59.96255405   413.27779219]
 [   66.43769647   634.38299912]
 [  287.54290339   627.9078567 ]
 [  281.06776098   406.80264977]
 [  274.59261856   185.69744285]
 [  268.12040343   -35.30780694]
 [  261.64526101  -256.41301386]
 [  255.1701186   -477.51822079]
 [  476.27532553  -483.99336321]
 [  482.75046794  -262.88815628]
 [  489.22561036   -41.78294935]
 [  495.69782549   179.22230043]
 [  502.17296791   400.32750736]
 [  508.64811032   621.43271429]
 [-1053.59

In [13]:
save_filename = os.path.join(data_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Shiwei\20210423-E14_Forebrain_smRNA\translated_positions_all.txt


# further adjust manually

In [14]:
adjust_before = np.array([[508.65,621.43],[-388.72, 205.12],[-1034.2, -1279.9]])

adjust_after = np.array([[411.3,606.1],[-484.8, 184.8],[-1127.2,-1300.9]])


delta_adjust = adjust_after - adjust_before
delta_adjust

array([[-97.35, -15.33],
       [-96.08, -20.32],
       [-93.  , -21.  ]])

In [16]:
manual_shift= np.mean(delta_adjust,axis=0)
manual_shift

array([-95.47666667, -18.88333333])

In [17]:
#manual_shift = np.array([-28.1,  -8.7])
adjusted_new_positions = new_positions + manual_shift

In [18]:
adj_save_filename = os.path.join(data_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

D:\Shiwei\20210423-E14_Forebrain_smRNA\adjusted_translated_positions_all.txt
